In [23]:
""" Implementing normal GAN for synthetic data experiment """

import numpy as np

from IPython.core.debugger import Tracer

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Activation, Lambda
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
import keras.backend as K

from sklearn.model_selection import train_test_split
from keras.utils import np_utils

import matplotlib.pyplot as plt


from tensorflow import set_random_seed
# for reproducibility 
np.random.seed(1)
set_random_seed(1)

In [30]:

class GAN(object):

    """ Generative Adversarial Network class """
    def __init__(self):

        self.x_combined_batch = None
        self.y_combined_batch = None
        self.number_of_classes = 2
        self.input_dim = 2
        self.gen_input_dim = 100
        self.epochs = 20000
        self.batch_size = 256
        self.save_interval = 100
        # There are 2 inlier circles
        # Inlier circle centers on the 1st axis; for all other dimensions, the centers are at 0.
        self.x1 = -3
        self.x2 = +3
        self.r = 1
        # Number of inlier samples for each class
        self.n = 1000
        # width of the outlier outer square side
        self.s = 100

        # Confidence loss weight
        self.beta = 0.0

        # get in-distribution data from 2 circles (2 classes)
        self.data, self.labels = self.get_data()

        print(self.data.shape, self.labels.shape)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data, self.labels, test_size=0.5, shuffle=True, stratify=self.labels,
                                                            random_state=1)

        self.Y_train = np_utils.to_categorical(self.y_train, self.number_of_classes)
        self.Y_test = np_utils.to_categorical(self.y_test, self.number_of_classes)

        self.optimizer = RMSprop(lr=0.00005) #, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss=self.wasserstein_loss, optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss=self.wasserstein_loss, optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()
        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)




    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)


    # return True if point p (a list) lies inside a hypersphere centered at the origin with radius r
    def is_in_hypersphere(self, r, p):
        p = np.array(p)
        return np.sum(p*p) <= r**2


    # return True if point p (a list) lies inside a hypersphere centered at the (x, 0, 0, ...) with radius r
    def is_in_hypersphere_centered_at(self, x, r, p):
        p = np.array(p)
        p[0] -= x
        return np.sum(p*p) <= r**2


    # uniformly sample a point from a square centered at the origin with side = s
    def sample_point_from_square(self, s):
        p = []
        # first point
        for i in range(0, self.input_dim):
            p.append(np.random.uniform(-1*s/2, s/2, 1))
        return p


    # sample points using rejection sampling from a sphere centered at the origin
    def sample_point_from_hypersphere(self, r):
        p = self.sample_point_from_square(2*r)
        while not self.is_in_hypersphere(r, p):
            p = self.sample_point_from_square(2*r)
        return p
            

    # sample from a hypersphre with rejection sampling
    def get_hypersphere_data(self, x, r, n):
        data = np.zeros((n, self.input_dim))
        count = 0
        while count<n:
            data[count, :] = self.sample_point_from_hypersphere(r)
            # update the point with offset for the 1st dimension
            data[count, 0] += x
            count += 1

        data = data.astype(np.float32)
        return data
        # plt.scatter(x, y, s = 4)
    
    
    def get_data(self):
        # class 0 data
        data = self.get_hypersphere_data(self.x1, self.r, self.n)
        labels = np.ones((self.n, 1))*0
        print(labels.shape)
        # class 1 data
        data = np.concatenate((data, self.get_hypersphere_data(self.x2, self.r, self.n)), axis=0)
        labels = np.concatenate((labels, np.ones((self.n, 1))*1), axis=0)
        print(labels.shape)

        return data.astype(np.float32), labels.astype(np.float32)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(self.gen_input_dim,)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.input_dim, activation='linear'))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Dense(256, input_shape=(self.input_dim, )))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1))
        model.summary()

        return model

    
    """
    def __generator(self):
        # Declare generator 

        model = Sequential()
        model.add(Dense(500, input_shape=(self.gen_input_dim,)))
        model.add(Activation('relu'))
        model.add(Dense(500))
        model.add(Activation('relu'))
        model.add(Dense(self.input_dim, activation='linear'))

        return model

    def __discriminator(self):
        # Declare discriminator

        model = Sequential()
        model.add(Dense(500, input_shape=(self.input_dim, )))
        model.add(Activation('relu'))
        model.add(Dense(500))
        model.add(Activation('relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model
    """


    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model


    def train(self):

        # gen_data = np.empty((self.epochs*10, self.input_dim))
        # count = 0
        y_discriminator_neg_ones = -1 * np.ones((self.batch_size/2, 1)).astype(np.float32)
        y_discriminator_ones = np.ones((self.batch_size/2, 1)).astype(np.float32)
        y_discriminator_zeros = np.zeros((self.batch_size/2, 1)).astype(np.float32)
        y_out = np.ones((self.batch_size/2, 2)).astype(np.float32)
        sample_weight = np.ones((self.batch_size)).astype(np.float32)
        sample_weight[self.batch_size/2:] = self.beta

        for cnt in range(self.epochs):

            ## train discriminator
            random_index = np.random.randint(0, len(self.X_train) - self.batch_size/2)
            legit_images = self.X_train[random_index : random_index + self.batch_size/2]

            gen_noise = np.random.normal(0, 1, (self.batch_size/2, self.gen_input_dim))
            synthetic_images = self.G.predict(gen_noise)

            self.x_combined_batch = np.concatenate((legit_images, synthetic_images))
            self.y_combined_batch = np.concatenate((y_discriminator_neg_ones, y_discriminator_ones))

            d_loss = self.D.train_on_batch(self.x_combined_batch, self.y_combined_batch)

            """
            gen_noise = np.random.normal(0, 1, (100, 100))
            syntetic_images = self.G.predict(gen_noise)
            preds = self.D.predict(syntetic_images)
            ind = np.where(preds < self.thresh)[0]
            if len(ind) > 0:
                gen_data[count:count+min(10, ind.shape[0]),] = syntetic_images[ind[0:min(10, ind.shape[0])],]
                count = count + min(10, ind.shape[0])
            """

            # train generator
            noise = np.random.normal(0, 1, (self.batch_size/2, self.gen_input_dim))
            g_loss = self.stacked_generator_discriminator.train_on_batch([noise], [y_discriminator_neg_ones])

            print('epoch: %d' % cnt)
            print(d_loss)
            print(g_loss)
            #print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f], [ Classifier :: loss: %f]' % 
            #       (cnt, d_loss[0], g_loss, 10.1))

            if cnt % self.save_interval == 0:
                self.plot_data(save2file=True, step=cnt)



    def plot_data(self, save2file, step=0):
        ''' Plot and generated images '''

        gen_noise = np.random.normal(0, 1, (self.X_test.shape[0], self.gen_input_dim))
        synthetic_images = self.G.predict(gen_noise)

        x = np.concatenate((self.X_test, synthetic_images), axis=0)
        #x = self.x_combined_batch
        # display a 2D plot of the digit classes in the latent space

        y = np.concatenate((self.y_test, 2 * np.ones((synthetic_images.shape[0], 1))))
        #y = self.y_combined_batch
        plt.figure(figsize=(12, 10))
        plt.scatter(x[:, 0], x[:, 1], c=y.squeeze())
        plt.colorbar()
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        if save2file:
            filename = "./images/mnist_%d.png" % step
            plt.savefig(filename)
        plt.close('all')
        #plt.show()



In [31]:
gan = GAN()
gan.train()

(1000, 1)
(2000, 1)
((2000, 2), (2000, 1))
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 256)               768       
_________________________________________________________________
batch_normalization_31 (Batc (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_49 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_32 (Batc (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_67 (Dense)             (Non

epoch: 161
[-11.244639, 0.00390625]
2.142548
epoch: 162
[-10.68286, 0.00390625]
2.4796188
epoch: 163
[-10.953952, 0.0078125]
4.470952
epoch: 164
[-11.00599, 0.01953125]
2.671876
epoch: 165
[-11.107005, 0.0078125]
3.111
epoch: 166
[-11.131155, 0.00390625]
2.1316428
epoch: 167
[-11.117372, 0.00390625]
4.497566
epoch: 168
[-11.328713, 0.00390625]
2.34015
epoch: 169
[-11.159352, 0.0078125]
0.6719246
epoch: 170
[-11.299521, 0.0078125]
1.1867595
epoch: 171
[-10.703755, 0.0234375]
1.3062761
epoch: 172
[-11.383391, 0.0078125]
3.6909344
epoch: 173
[-11.295718, 0.0]
0.97860366
epoch: 174
[-11.205722, 0.0]
3.0156155
epoch: 175
[-11.382327, 0.00390625]
2.1362731
epoch: 176
[-11.239747, 0.0]
3.3518274
epoch: 177
[-11.425668, 0.0078125]
2.3783855
epoch: 178
[-11.465423, 0.00390625]
2.2539878
epoch: 179
[-11.327871, 0.0078125]
2.0653296
epoch: 180
[-11.274228, 0.0078125]
1.1999866
epoch: 181
[-11.652444, 0.0]
3.062921
epoch: 182
[-11.193845, 0.00390625]
2.9044776
epoch: 183
[-11.375834, 0.00390625]
0

epoch: 363
[-13.710114, 0.00390625]
2.348359
epoch: 364
[-13.831885, 0.0]
1.3951976
epoch: 365
[-13.869384, 0.01171875]
2.3822818
epoch: 366
[-13.652832, 0.00390625]
2.143342
epoch: 367
[-13.4095745, 0.0078125]
3.283999
epoch: 368
[-13.552064, 0.00390625]
0.82824445
epoch: 369
[-13.685293, 0.0]
1.2812536
epoch: 370
[-13.805117, 0.0]
4.1331124
epoch: 371
[-13.864524, 0.0]
2.3121247
epoch: 372
[-13.691537, 0.0078125]
1.8397318
epoch: 373
[-13.600892, 0.00390625]
2.4759817
epoch: 374
[-13.669854, 0.0]
2.8614671
epoch: 375
[-13.840461, 0.01171875]
1.930454
epoch: 376
[-13.754851, 0.0078125]
0.17410117
epoch: 377
[-13.889487, 0.00390625]
2.3849497
epoch: 378
[-13.665719, 0.00390625]
2.89995
epoch: 379
[-13.353995, 0.01171875]
2.1377885
epoch: 380
[-13.923728, 0.0078125]
0.7394337
epoch: 381
[-13.45864, 0.0078125]
1.5172765
epoch: 382
[-13.536862, 0.0078125]
-0.44437572
epoch: 383
[-14.155417, 0.00390625]
4.289877
epoch: 384
[-13.74594, 0.0]
1.369456
epoch: 385
[-13.887488, 0.0078125]
1.3034

epoch: 563
[-15.249474, 0.00390625]
1.7549934
epoch: 564
[-15.609313, 0.0]
0.76264817
epoch: 565
[-15.538118, 0.00390625]
4.897496
epoch: 566
[-14.619204, 0.0]
0.8881586
epoch: 567
[-15.364174, 0.0]
2.1187835
epoch: 568
[-15.670747, 0.01171875]
-0.52359986
epoch: 569
[-15.533165, 0.0078125]
1.2450156
epoch: 570
[-14.971865, 0.0]
1.0639598
epoch: 571
[-15.272921, 0.0]
0.9530568
epoch: 572
[-15.423864, 0.0]
1.7810984
epoch: 573
[-15.048524, 0.00390625]
1.1573157
epoch: 574
[-15.873628, 0.0]
0.87132025
epoch: 575
[-15.691887, 0.0078125]
0.080423474
epoch: 576
[-15.361871, 0.00390625]
2.2580009
epoch: 577
[-15.704369, 0.0]
1.5079558
epoch: 578
[-15.472994, 0.01171875]
0.007866263
epoch: 579
[-15.212669, 0.0]
1.6627386
epoch: 580
[-14.7958975, 0.0078125]
0.19635737
epoch: 581
[-15.189827, 0.0078125]
1.0064292
epoch: 582
[-15.367617, 0.00390625]
1.6119108
epoch: 583
[-15.069649, 0.01171875]
-0.2495974
epoch: 584
[-15.097925, 0.0]
0.06154996
epoch: 585
[-15.47821, 0.0]
0.12830678
epoch: 586
[

epoch: 762
[-17.001186, 0.00390625]
0.5257447
epoch: 763
[-16.825188, 0.0]
3.3831728
epoch: 764
[-16.711332, 0.0078125]
1.1090093
epoch: 765
[-17.115181, 0.0]
0.66350293
epoch: 766
[-16.768038, 0.0]
2.1815643
epoch: 767
[-16.761286, 0.0078125]
2.147296
epoch: 768
[-16.878517, 0.0]
1.132934
epoch: 769
[-16.602446, 0.00390625]
0.24002445
epoch: 770
[-16.43085, 0.00390625]
0.6219824
epoch: 771
[-16.653526, 0.00390625]
1.2990339
epoch: 772
[-17.202785, 0.00390625]
1.4127711
epoch: 773
[-17.030266, 0.01171875]
0.25931942
epoch: 774
[-16.872261, 0.0]
0.39017963
epoch: 775
[-16.314522, 0.0078125]
1.3999006
epoch: 776
[-16.557402, 0.00390625]
0.89532906
epoch: 777
[-16.841835, 0.0]
-0.23277766
epoch: 778
[-17.231138, 0.0]
0.86352557
epoch: 779
[-16.379066, 0.00390625]
0.2074486
epoch: 780
[-17.54669, 0.0]
-0.18614918
epoch: 781
[-16.813545, 0.0078125]
0.16926551
epoch: 782
[-16.970339, 0.01171875]
0.3828892
epoch: 783
[-17.114317, 0.0]
0.29115796
epoch: 784
[-16.697231, 0.0]
0.86812675
epoch: 

...

epoch: 12248
[-149.29326, 0.0]
3.7580905
epoch: 12249
[-156.50883, 0.0]
3.1714811
epoch: 12250
[-144.1714, 0.0]
2.6330233
epoch: 12251
[-139.982, 0.0]
1.8829787
epoch: 12252
[-148.1538, 0.0]
3.7580907
epoch: 12253
[-148.8406, 0.00390625]
4.133113
epoch: 12254
[-165.04865, 0.0]
3.0080457
epoch: 12255
[-155.81883, 0.0]
4.508136
epoch: 12256
[-154.72559, 0.0]
3.383068
epoch: 12257
[-153.24849, 0.0]
4.234203
epoch: 12258
[-159.06834, 0.0]
3.0080462
epoch: 12259
[-153.36623, 0.0]
3.0080457
epoch: 12260
[-149.36784, 0.0]
2.2580009
epoch: 12261
[-152.3877, 0.00390625]
4.133113
epoch: 12262
[-148.99968, 0.0]
4.508136
epoch: 12263
[-149.77104, 0.0]
2.2580013
epoch: 12264
[-156.73927, 0.0]
4.133113
epoch: 12265
[-145.30617, 0.00390625]
5.2581797
epoch: 12266
[-141.67145, 0.0]
4.2903657
epoch: 12267
[-139.84212, 0.0]
2.633023
epoch: 12268
[-167.69331, 0.0]
3.7580905
epoch: 12269
[-145.97174, 0.0]
2.2580009
epoch: 12270
[-156.01187, 0.00390625]
4.133113
epoch: 12271
[-147.04118, 0.0]
4.508135
epoc

epoch: 12448
[-156.74121, 0.0]
3.0080457
epoch: 12449
[-150.98488, 0.0]
2.6330237
epoch: 12450
[-167.88995, 0.0]
3.383068
epoch: 12451
[-163.79462, 0.0]
3.5319555
epoch: 12452
[-147.01613, 0.00390625]
3.882474
epoch: 12453
[-112.3305, 0.0]
3.0080457
epoch: 12454
[-150.91228, 0.0]
2.2580009
epoch: 12455
[-163.61163, 0.0]
2.6330233
epoch: 12456
[-152.79926, 0.0]
1.9836493
epoch: 12457
[-158.36545, 0.00390625]
2.6330233
epoch: 12458
[-154.6392, 0.0]
1.1329341
epoch: 12459
[-158.11769, 0.0]
3.5111353
epoch: 12460
[-150.33055, 0.0]
3.383068
epoch: 12461
[-155.26128, 0.0]
1.8829784
epoch: 12462
[-167.59966, 0.0]
2.6330235
epoch: 12463
[-168.12944, 0.0]
4.133114
epoch: 12464
[-180.5205, 0.0]
3.3830686
epoch: 12465
[-148.48067, 0.0]
3.008046
epoch: 12466
[-154.72578, 0.0]
2.2580013
epoch: 12467
[-157.28592, 0.0]
3.7580903
epoch: 12468
[-159.43065, 0.0]
2.6330233
epoch: 12469
[-164.32613, 0.0]
4.133113
epoch: 12470
[-175.97945, 0.0]
4.1331124
epoch: 12471
[-166.4824, 0.0]
2.2580013
epoch: 12472

epoch: 12648
[-157.65121, 0.0]
1.5079565
epoch: 12649
[-160.21196, 0.0]
-0.36715567
epoch: 12650
[-169.10152, 0.00390625]
3.383068
epoch: 12651
[-167.34673, 0.0]
1.5079564
epoch: 12652
[-152.73306, 0.0]
2.2580013
epoch: 12653
[-166.61763, 0.0]
1.882978
epoch: 12654
[-166.9683, 0.00390625]
2.2580009
epoch: 12655
[-151.5861, 0.00390625]
4.7797346
epoch: 12656
[-154.01411, 0.00390625]
4.5081353
epoch: 12657
[-149.03955, 0.0]
2.407245
epoch: 12658
[-163.75473, 0.0]
3.5023372
epoch: 12659
[-165.70782, 0.0]
1.662425
epoch: 12660
[-131.40845, 0.00390625]
4.8831573
epoch: 12661
[-174.35884, 0.0]
4.508135
epoch: 12662
[-142.33148, 0.0]
4.8831573
epoch: 12663
[-138.97557, 0.0]
3.3830678
epoch: 12664
[-152.57793, 0.0]
3.0080457
epoch: 12665
[-160.91571, 0.0]
3.0080457
epoch: 12666
[-172.38622, 0.0]
2.0021274
epoch: 12667
[-162.8963, 0.0]
4.2559175
epoch: 12668
[-166.5449, 0.0]
3.3830686
epoch: 12669
[-161.58856, 0.00390625]
1.8829786
epoch: 12670
[-164.44728, 0.00390625]
2.7528481
epoch: 12671
[-

epoch: 12847
[-177.65968, 0.0]
3.383068
epoch: 12848
[-154.09235, 0.00390625]
5.2581797
epoch: 12849
[-156.5827, 0.0]
3.7580905
epoch: 12850
[-165.7884, 0.00390625]
3.7580905
epoch: 12851
[-155.96095, 0.00390625]
4.8831577
epoch: 12852
[-171.15999, 0.0]
2.6330237
epoch: 12853
[-160.48077, 0.0]
2.6330233
epoch: 12854
[-182.8268, 0.0]
4.8831577
epoch: 12855
[-163.6103, 0.00390625]
4.8831573
epoch: 12856
[-146.15938, 0.0]
3.383068
epoch: 12857
[-169.84615, 0.0]
2.258001
epoch: 12858
[-158.5246, 0.0]
2.6330233
epoch: 12859
[-174.61525, 0.0]
1.8829784
epoch: 12860
[-166.52495, 0.0]
4.883157
epoch: 12861
[-171.7773, 0.0]
0.75791156
epoch: 12862
[-152.94481, 0.0]
3.0080462
epoch: 12863
[-181.41066, 0.0]
0.75791144
epoch: 12864
[-176.28029, 0.0]
4.1331124
epoch: 12865
[-170.36296, 0.0]
3.3830683
epoch: 12866
[-161.91583, 0.0]
2.43821
epoch: 12867
[-168.83647, 0.0]
1.8829786
epoch: 12868
[-164.45782, 0.0]
2.6330235
epoch: 12869
[-157.21877, 0.0]
2.6330235
epoch: 12870
[-166.32182, 0.0]
0.869363

KeyboardInterrupt: 